In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [2]:
# Step 1: Initialize parameters
np.random.seed(42)  # For reproducibility

# True demand parameters (unknown to the algorithm)
true_a = 10  # Intercept
true_b = 1    # Slope
noise_std = 2#0.5  # Standard deviation of noise
p_min = 0  # Minimum feasible price
p_max = 25 # Maximum feasible price

# Predefined prices for periods 1 and 2
prices = [4, 5]
demands = []
revenues = []      # Noisy revenues
det_revenues = []  # Deterministic (noise-free) revenues
regrets = []       # Instantaneous regret
cumulative_regret = [0]  # Initialize cumulative regret

In [3]:
# Demand function with noise
def demand_function(price):
    """Simulate demand given a price with noise."""
    noise = np.random.normal(0, noise_std)  # Random noise
    demand = true_a - true_b * price + noise
    return max(0, demand)  # Demand cannot be negative

# Deterministic demand function (noise-free)
def deterministic_demand(price):
    """Simulate demand without noise."""
    demand = true_a - true_b * price
    return max(0, demand)  # Demand cannot be negative

# Step 2: Calculate the true optimal price and revenue
optimal_price = true_a / (2 * true_b)  # Revenue-maximizing price
optimal_revenue = optimal_price * deterministic_demand(optimal_price)  # Optimal revenue
print(f"Optimal Price: {optimal_price:.2f}, Optimal Revenue: {optimal_revenue:.2f}")

# Simulate demand and revenue for initial periods
for p in prices:
    noisy_demand = demand_function(p)
    demands.append(noisy_demand)
    revenues.append(p * noisy_demand)
    det_revenues.append(p * deterministic_demand(p))
    regrets.append(optimal_revenue - det_revenues[-1])
    cumulative_regret.append(cumulative_regret[-1] + regrets[-1])

# Step 3: Estimate demand function and find optimal prices iteratively
n_periods = 5000  # Total number of periods
for t in range(2, n_periods):
    # Prepare data for OLS
    X = np.array(prices).reshape(-1, 1)  # Reshape to 2D for regression
    y = np.array(demands)

    # Fit OLS regression
    model = LinearRegression()
    model.fit(X, y)

    # Extract coefficients
    est_a = model.intercept_
    est_b = -model.coef_[0]  # Negative because demand decreases with price

       # Calculate the best price for revenue maximization
    psi_t = est_a / (2 * est_b)
    p_t = max(p_min, min(p_max, psi_t))  # Ensure price is within feasible range


     # Simulate demand for this price
    noisy_demand = demand_function(p_t)
    noise_free_demand = deterministic_demand(p_t)

    # Append results
    prices.append(p_t)
    demands.append(noisy_demand)
    revenues.append(p_t * noisy_demand)
    det_revenues.append(p_t * noise_free_demand)

    # Calculate regret
    instantaneous_regret = optimal_revenue - det_revenues[-1]
    regrets.append(instantaneous_regret)
    cumulative_regret.append(cumulative_regret[-1] + instantaneous_regret)

    # Print results for the current period
    print(f"Period {t+1}: Price = {p_t:.2f}, Noisy Demand = {noisy_demand:.2f}, "
          f"Deterministic Revenue = {det_revenues[-1]:.2f}, "
          f"Instantaneous Regret = {instantaneous_regret:.2f}")

# Final results
results = pd.DataFrame({
    'Period': range(1, n_periods + 1),
    'Price': prices,
    'Noisy Revenue': revenues,
    'Noise-Free Revenue': det_revenues,
    'Instantaneous Regret': regrets,
    'Cumulative Regret': cumulative_regret[1:]  # Skip the initial zero
})

Optimal Price: 5.00, Optimal Revenue: 25.00
Period 3: Price = 3.54, Noisy Demand = 7.75, Deterministic Revenue = 22.87, Instantaneous Regret = 2.13
Period 4: Price = 3.63, Noisy Demand = 9.42, Deterministic Revenue = 23.12, Instantaneous Regret = 1.88
Period 5: Price = 3.38, Noisy Demand = 6.15, Deterministic Revenue = 22.39, Instantaneous Regret = 2.61
Period 6: Price = 3.93, Noisy Demand = 5.60, Deterministic Revenue = 23.86, Instantaneous Regret = 1.14
Period 7: Price = 3.85, Noisy Demand = 9.31, Deterministic Revenue = 23.68, Instantaneous Regret = 1.32
Period 8: Price = 3.87, Noisy Demand = 7.67, Deterministic Revenue = 23.72, Instantaneous Regret = 1.28
Period 9: Price = 3.87, Noisy Demand = 5.19, Deterministic Revenue = 23.73, Instantaneous Regret = 1.27
Period 10: Price = 3.84, Noisy Demand = 7.24, Deterministic Revenue = 23.66, Instantaneous Regret = 1.34
Period 11: Price = 3.84, Noisy Demand = 5.23, Deterministic Revenue = 23.66, Instantaneous Regret = 1.34
Period 12: Price =